In [1]:
import pandas as pd
df = pd.read_csv('../case_data/comment.csv')

In [2]:
import pandas as pd
import numpy as np
import os
import jieba 
import jieba.posseg as pseg
def my_cut(text): 
    
#     word_dict_file = './sport_word.dict'
#     # 加载自定义词典
#     jieba.load_userdict(word_dict_file)
 
        
    # 加载停用词
    stop_words = [] 
    with open("../stopwords.txt", encoding='utf-8') as f:
       lines = f.readlines()
       for line in lines:
           stop_words.append(line.strip())
    # stop_words[:10]
           
    return [w for w,flag in pseg.cut(text) if w not in stop_words and flag in ['n']]

# 字符串存储

In [3]:
def str2csv(filePath, s, x):
    '''
    将字符串写入到本地csv文件中
    :param filePath: csv文件路径
    :param s: 待写入字符串(逗号分隔格式)
    '''
    if x=='node':
        with open(filePath, 'w', encoding='utf-8') as f:
            f.write("Label,Weight\r")
            f.write(s)
        print('写入文件成功,请在'+filePath+'中查看')
    else:
        with open(filePath, 'w', encoding='utf-8') as f:
            f.write("Source,Target,Weight\r")
            f.write(s)
        print('写入文件成功,请在'+filePath+'中查看')

## 构建字典

In [4]:
def sortDictValue(dict, is_reverse):
    '''
    将字典按照value排序
    :param dict: 待排序的字典
    :param is_reverse: 是否按照倒序排序
    :return s: 符合csv逗号分隔格式的字符串
    '''
    # 对字典的值进行倒序排序,items()将字典的每个键值对转化为一个元组,key输入的是函数,item[1]表示元组的第二个元素,reverse为真表示倒序
    tups = sorted(dict.items(), key=lambda item: item[1], reverse=is_reverse)
    s = ''
    for tup in tups:  # 合并成csv需要的逗号分隔格式
        s = s + tup[0] + ',' + str(tup[1]) + '\n'
    return s

## 构建共现矩阵

In [5]:
def build_matrix(co_authors_list, is_reverse):
    '''
    根据共同列表,构建共现矩阵(存储到字典中),并将该字典按照权值排序
    :param co_authors_list: 共同列表
    :param is_reverse: 排序是否倒序
    :return node_str: 三元组形式的节点字符串(且符合csv逗号分隔格式)
    :return edge_str: 三元组形式的边字符串(且符合csv逗号分隔格式)
    '''
    node_dict = {}  # 节点字典,包含节点名+节点权值(频数)
    edge_dict = {}  # 边字典,包含起点+目标点+边权值(频数)
    # 第1层循环,遍历整表的每行信息
    for row_authors in co_authors_list:
        row_authors_list = row_authors.split(' ') # 依据','分割每行,存储到列表中
        # 第2层循环
        for index, pre_au in enumerate(row_authors_list): # 使用enumerate()以获取遍历次数index
            # 统计单个词出现的频次
            if pre_au not in node_dict:
                node_dict[pre_au] = 1
            else:
                node_dict[pre_au] += 1
            # 若遍历到倒数第一个元素,则无需记录关系,结束循环即可
            if pre_au == row_authors_list[-1]:
                break
            connect_list = row_authors_list[index+1:]
            # 第3层循环,遍历当前行词后面所有的词,以统计两两词出现的频次
            for next_au in connect_list:
                A, B = pre_au, next_au
                # 固定两两词的顺序
                # 仅计算上半个矩阵
                if A==B:
                    continue
                if A > B:
                    A, B = B, A
                key = A+','+B  # 格式化为逗号分隔A,B形式,作为字典的键
                # 若该关系不在字典中,则初始化为1,表示词间的共同出现次数
                if key not in edge_dict:
                    edge_dict[key] = 1
                else:
                    edge_dict[key] += 1
    # 对得到的字典按照value进行排序
    node_str = sortDictValue(node_dict, is_reverse)  # 节点
    edge_str = sortDictValue(edge_dict, is_reverse)   # 边
    return node_str, edge_str

In [22]:
from itertools import chain
# os.chdir(r'.\')#os.chdir() 方法用于改变当前工作目录到指定的路径
filePath1 = r'./data/node.csv'
filePath2 = r'./data/edge.csv'
# 读取csv文件获取数据并存储到列表中
df = pd.read_csv('../case_data/comment.csv',encoding='utf-8')
co_ist = pd.read_csv('./data/words.csv')
# co_ist = co_ist.values
b = co_ist.values
co_ist = list(chain.from_iterable(b))

# df_ = df['content'][:100]
# co_ist = [ " ".join(my_cut(str(w))) for w in df_ if len(w)>0] 
# print(co_ist)
# 根据共同词列表, 构建共现矩阵(存储到字典中), 并将该字典按照权值排序
node_str, edge_str = build_matrix(co_ist, is_reverse=True)
#print(edge_str)
# 将字符串写入到本地csv文件中
str2csv(filePath1,node_str,'node')
str2csv(filePath2,edge_str,'edge')

写入文件成功,请在./data/node.csv中查看
写入文件成功,请在./data/edge.csv中查看


In [7]:
# df = pd.read_csv('../case_data/comment.csv',encoding='utf-8')
# df_ = df['content']
# co_ist = [" ".join(my_cut(str(w))) for w in df_] 
# co_ist = [item for item in co_ist if len(item)>0]
# df2 = pd.DataFrame(co_ist)
# df2.to_csv('./data/words.csv', index=False)

In [4]:
import pandas as pd
edge_str = pd.read_csv('./data/edge.csv',encoding='utf-8')
edge_str.shape
 
edge_str1 = edge_str[edge_str['Weight']>300]
edge_str1.shape
 
Source = edge_str1['Source'].tolist()
Target = edge_str1['Target'].tolist()
co = Source + Target
co =list(set(co))
 
node_str = pd.read_csv('./data/node.csv',encoding='utf-8')
#node_str
 
node_str=node_str[node_str['Label'].isin(co)]
node_str['id']=node_str['Label']
node_str = node_str[['id','Label','Weight']] # 调整列顺序
#node_str
 
node_str.to_csv(path_or_buf="./data/node300.csv", index=False) # 写入csv文件
edge_str1.to_csv(path_or_buf="./data/edge300.csv", index=False) # 写入csv文件